In [25]:
import pandas as pd
import numpy as np

In [26]:
def clean_dataframe(df, registry, start, end, verbose=False):

    registry['DateStop'] = pd.to_datetime(registry['DateStop'])
    registry = registry[registry['Pollutant'] == pollutant]
    active = registry[(pd.isna(registry.DateStop)) | (registry.DateStop > pd.to_datetime(end))]

    df['Date'] = pd.to_datetime(df['Date'])

    data = df[(df.Date > pd.to_datetime(start)) & (df.Date < pd.to_datetime(end))]
    data = data.groupby('IDStation', as_index=False).apply(lambda g: g.mean(skipna=True))
    data['IDStation'] = data['IDStation'].astype(int)


    active = registry[(pd.isna(registry.DateStop)) | (registry.DateStop > pd.to_datetime(end))]

    clean_df = pd.merge(data, active[['IDStation', 'Latitude', 'Longitude', 'Altitude']], on='IDStation', how='inner')

    if verbose:
        print(clean_df.shape)
        print(clean_df.info())        

    return clean_df

In [43]:
def normalize(df, feature, verbose=False, output=None):

    min_value = df[feature].min()
    max_value = df[feature].max()

    if verbose:
        print('\n' + feature)
        print('Min: {}, Max: {}'.format(min_value, max_value))
    
    if output:
        output.write('\n' + feature + '\n')
        output.write('Min: {}, Max: {}\n'.format(min_value, max_value))
    
    normalized = df[feature].copy()

    if min_value < 0:

        normalized += np.abs(min_value)
        max_value += np.abs(min_value)

        if verbose: 
            print('Some negative values found, with ratio: {}\n'.format(round(100*(np.abs(min_value) / max_value), 2)))

        if output:
            output.write('Some negative values found, with ratio: {}\n'.format(round(100*(np.abs(min_value) / max_value), 2)))
    else:

        if verbose: print('Only positive values found.\n')

        if output:  output.write('Only positive values found.\n')
  
    df[feature + '_Norm'] = normalized / max_value 

    if verbose: print(df.head(3))

    return df

In [44]:
pollutants = ['NO2', 'NOx', 'PM2.5', 'PM10']

In [45]:
registry = pd.read_csv('./data/registry.csv')

In [46]:
download = True

In [49]:
with open('./data/report.txt', 'w') as f:

    for pollutant in pollutants:

        df = pd.read_csv('./data/raw/{}_sit_monthly.csv'.format(pollutant)).drop('Unnamed: 0', axis=1)

        clean_df_2019 = clean_dataframe(df, registry, 'March 2019', 'July 2019', verbose=False)

        if download: clean_df_2019.to_csv('./data/2019/{}_2019.csv'.format(pollutant))

        clean_df_2020 = clean_dataframe(df, registry, 'March 2020', 'July 2020', verbose=False)

        if download: clean_df_2020.to_csv('./data/2020/{}_2020.csv'.format(pollutant))

        diff_df = pd.merge(clean_df_2019, clean_df_2020, on=['IDStation', 'Latitude', 'Longitude', 'Altitude'], suffixes=['_2019', '_2020'])

        diff_df['{}_Delta'.format(pollutant)] = diff_df[pollutant+ '_2019'] - diff_df[pollutant + '_2020']

        diff_df = diff_df[['IDStation', pollutant+ '_2019', pollutant+ '_2020', pollutant+ '_Delta', 'Latitude', 'Longitude']]

        diff_df = diff_df.dropna()  

        diff_df['{}_Abs'.format(pollutant)] = np.abs(diff_df['{}_Delta'.format(pollutant)])      

        for feature in [pollutant + '_2019', pollutant + '_2020', pollutant + '_Delta']:

            normalized_df = normalize(diff_df, feature, verbose=True, output=f)

        
        if download: diff_df.to_csv('./data/{}.csv'.format(pollutant))        


NO2_2019
Min: 7.619641537167014, Max: 62.5053218723954
Only positive values found.

   IDStation   NO2_2019   NO2_2020  NO2_Delta   Latitude  Longitude  \
0        501  45.858607  32.160306  13.698301  45.496316   9.190934   
1        504  31.963952  22.736508   9.227443  45.534766   9.236108   
2        514  26.261408  22.180302   4.081106  45.523426   9.044600   

     NO2_Abs  NO2_2019_Norm  
0  13.698301       0.733675  
1   9.227443       0.511380  
2   4.081106       0.420147  

NO2_2020
Min: 5.25955393886691, Max: 32.160305598077166
Only positive values found.

   IDStation   NO2_2019   NO2_2020  NO2_Delta   Latitude  Longitude  \
0        501  45.858607  32.160306  13.698301  45.496316   9.190934   
1        504  31.963952  22.736508   9.227443  45.534766   9.236108   
2        514  26.261408  22.180302   4.081106  45.523426   9.044600   

     NO2_Abs  NO2_2019_Norm  NO2_2020_Norm  
0  13.698301       0.733675       1.000000  
1   9.227443       0.511380       0.706974  
2   